In [1]:
import os, pandas as pd
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import cudf, glob, gc, pickle

VER = 162
POSTFIX = '_LB'

DAY = 0

In [2]:
files = glob.glob('../../data/infer_data/test_parquet/*')
len( files )

17

In [3]:
df = pd.read_parquet(files[0])

In [4]:
df.head()

,session,aid,ts,type
0,14199779,1747653,1662205325650,clicks
1,14199780,295604,1662205326327,clicks
2,14199781,404474,1662205326879,clicks
3,14199781,939160,1662205330528,clicks
4,14199782,1477301,1662205327444,clicks


In [5]:
type_map = {'clicks':0, 'carts':1, 'orders':2}

In [6]:
files[:4]

['/raid/Kaggle/otto/parquet/test_parquet/001300000_001400000.parquet',
 '/raid/Kaggle/otto/parquet/test_parquet/000800000_000900000.parquet',
 '/raid/Kaggle/otto/parquet/test_parquet/001500000_001600000.parquet',
 '/raid/Kaggle/otto/parquet/test_parquet/000000000_000100000.parquet']

In [7]:
#type_weight = {0:0, 1:1, 2:1}

In [8]:
#MN = 1661119200

In [9]:
%%time
PIECES = 1
SIZE = 1.86e6/PIECES

# COMPUTE IN PARTS FOR MEMORY MANGEMENT
for PART in range(PIECES):
    print()
    print('### PART',PART+1)
    
    # MERGE IS FASTEST PROCESSING CHUNKS WITHIN CHUNKS
    # => OUTER CHUNKS
    for a,b in [(0,17)]: #,(20,40),(40,60),(60,80),(80,100),(100,120)]:
        print(f'Processing {b-a} files...')
        
        # => INNER CHUNKS
        for k in range(a,b):
            # READ FILE
            df = cudf.read_parquet(files[k])
            df.ts = (df.ts//1000).astype('int32')
            df['type'] = df['type'].map(type_map)
            
            #df = df.loc[df.ts>1662328791 - 60*60*24*28]
            #df = df.loc[( df.ts >= MN + 60*60*24*DAY )] # START ON DAY OF WEEK 
            #if len(df)==0: continue
                
            df = df.sort_values(['session','ts'],ascending=[True,False])

            # CREATE PAIRS
            df = df.merge(df.loc[df['type'].isin([1,2])],on='session')
            df = df.loc[ ((df.ts_y - df.ts_x)> 0) ] #& ( df.ts_x < MN + 60*60*24*(DAY+1) ) ] 
            # MEMORY MANAGEMENT COMPUTE IN PARTS
            df = df.loc[(df.aid_x >= PART*SIZE)&(df.aid_x < (PART+1)*SIZE)]
            # ASSIGN WEIGHTS
            df = df[['session', 'aid_x', 'aid_y','ts_y','ts_x']].drop_duplicates(['session', 'aid_x', 'aid_y'])
            w = (1/2)**((df.ts_y - df.ts_x)/60/60/6) # 6 HOUR HALF LIFE
            df['wgt'] = w #df.type_y.map(type_weight)*w
            
            df = df[['aid_x','aid_y','wgt']]

            df.wgt = df.wgt.astype('float32')
            df = df.groupby(['aid_x','aid_y']).wgt.sum()
            # COMBINE INNER CHUNKS
            if k==a: tmp2 = df
            else: tmp2 = tmp2.add(df, fill_value=0)
            print(k,', ',end='')
        print()
        # COMBINE OUTER CHUNKS
        if a==0: tmp = tmp2
        else: tmp = tmp.add(tmp2, fill_value=0)
        del tmp2, df
        gc.collect()
    # CONVERT MATRIX TO DICTIONARY
    tmp = tmp.reset_index()
    tmp = tmp.sort_values(['aid_x','wgt'],ascending=[True,False])
    # SAVE TOP 40
    tmp = tmp.reset_index(drop=True)
    tmp['n'] = tmp.groupby('aid_x').aid_y.cumcount()
    tmp = tmp.loc[tmp.n<40].drop('n',axis=1)
    # SAVE PART TO DISK
    df = tmp.to_pandas().groupby('aid_x').aid_y.apply(list)
    with open(f'../../data/covisit_matrices/top_40_aids_v{VER}_d{DAY}_{PART}{POSTFIX}.pkl', 'wb') as f:
        pickle.dump(df.to_dict(), f)
        
    ## SAVE PART TO DISK
    #df = tmp.to_pandas().groupby('aid_x').wgt.apply(list)
    #with open(f'top_40_aids_v{VER}_{PART}_w.pkl', 'wb') as f:
    #    pickle.dump(df.to_dict(), f)


### PART 1
Processing 17 files...
0 , 1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 , 14 , 15 , 16 , 
CPU times: user 9.31 s, sys: 2.04 s, total: 11.4 s
Wall time: 12.2 s
